# Tests with the EVCXR kernel for Rust

In [34]:
:help

:clear,"Clear all state, keeping compilation cache"
:dep,"Add dependency. e.g. :dep regex = ""1.0"""
:efmt,Set the formatter for errors returned by ?
:explain,Print explanation of last error
:fmt,Set output formatter (default: {:?})
:help,Print command help
:internal_debug,Toggle various internal debugging code
:last_compile_dir,Print the directory in which we last compiled
:last_error_json,Print the last compilation error as JSON (for debugging)
:linker,"Set/print linker. Supported: system, lld, mold"
:load_config,Reloads startup configuration files. Accepts optional flag `--quiet` to suppress logging.


In [35]:
:dep showata = { version = "0.3.2", features = ["show_ndarray"]}
:dep ndarray = {version = "0.15.6", features = ["rayon"]}
:dep ndarray-linalg = {version = "0.16.0", features = ["openblas-system"]}
// :dep rayon = "1.7.0"

In [38]:
use showata::Showable;
// use ndarray::{prelude::*,Zip};
use ndarray::parallel::prelude::*;
// use ndarray::parallel::prelude::
// use ndarray::parallel::impl_par_methods::Zip::*;
use std::sync::Mutex;

In [39]:
let a = array![[1., 2.], [3., 4.]];
// a

Zip::indexed(&a).into_par_iter().for_each(|((idx1,idx2), x)| {
    println!("{},{}: {}", idx1,idx2, x);
});

0,0: 1
1,0: 3
0,1: 2
1,1: 4


In [62]:
let mut dummy_matr = Array2::<f64>::zeros((10,10));
let mut S_matr = Array2::<f64>::zeros((10,10));
let vec1 = vec![1.,2.,3.,4.,5.,6.,7.,8.,9.,10.];
let vec2 = vec![1.,2.,3.,4.,5.,6.,7.,8.,9.,10.];

Zip::indexed(&dummy_matr).par_map_assign_into(&mut S_matr, |(i,j), x| {
    if i == j {
       1. 
    } else {
        (i+j) as f64
    }
});

// S_matr.show();
// Zip::from(&vec1).and(&vec2).par_map_assign_into(&mut S_matr, |x, y| {
//     if x == y {
//         1.
//     } else {
//         0.
//     }
// });

// Zip::indexed(&mut S_matr).par_map_assign_into(&mut S_matr, |(i,j), x| {
//     if i == j {
//        1. 
//     } else {
//         (i+j) as f64
//     }
// });

// Zip::par_map_assign_into(&mut S_matr, &a, |x, y| {
//     *x = *y;
// });

In [63]:
S_matr

[[1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 [1.0, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 [2.0, 3.0, 1.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0],
 [3.0, 4.0, 5.0, 1.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0],
 [4.0, 5.0, 6.0, 7.0, 1.0, 9.0, 10.0, 11.0, 12.0, 13.0],
 [5.0, 6.0, 7.0, 8.0, 9.0, 1.0, 11.0, 12.0, 13.0, 14.0],
 [6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 1.0, 13.0, 14.0, 15.0],
 [7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 1.0, 15.0, 16.0],
 [8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 1.0, 17.0],
 [9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 1.0]], shape=[10, 10], strides=[10, 1], layout=Cc (0x5), const ndim=2

In [ ]:
let C_matr_AO_basis = Array2::<f64>::zeros((10,10));
let D_matr_mutex = Mutex::new(Array2::<f64>::zeros((10,10)));

Zip::indexed(C_matr_AO_basis.axis_iter(Axis(0))).par_for_each(|mu, row1| {
    Zip::indexed(C_matr_AO_basis.outer_iter()).par_for_each(|nu, row2| {
        let mut d = D_matr_mutex.lock().unwrap();
        let slice1 = row1.slice(s![..5]);
        let slice2 = row2.slice(s![..5]);
        d[(mu, nu)] = slice1.dot(&slice2);
    });
});

let D_matr = D_matr_mutex.into_inner().unwrap();
D_matr

While processing instruction `Ok("LOAD_AND_RUN /tmp/.tmpfm4H0t/target/x86_64-unknown-linux-gnu/debug/deps/libcode_10.so run_user_code_7")`, got error: Message("/tmp/.tmpfm4H0t/target/x86_64-unknown-linux-gnu/debug/deps/libcode_10.so: undefined symbol: cblas_ddot")


Error: Subprocess terminated with status: exit status: 99